# 146. LRU Cache

Design a data structure that follows the constraints of a Least Recently Used (LRU) cache.

Implement the LRUCache class:

LRUCache(int capacity) Initialize the LRU cache with positive size capacity.
int get(int key) Return the value of the key if the key exists, otherwise return -1.
void put(int key, int value) Update the value of the key if the key exists. Otherwise, add the key-value pair to the cache. If the number of keys exceeds the capacity from this operation, evict the least recently used key.
The functions get and put must each run in O(1) average time complexity.



Example 1:

Input
["LRUCache", "put", "put", "get", "put", "get", "put", "get", "get", "get"]
[[2], [1, 1], [2, 2], [1], [3, 3], [2], [4, 4], [1], [3], [4]]
Output
[null, null, null, 1, null, -1, null, -1, 3, 4]

Explanation
LRUCache lRUCache = new LRUCache(2);
lRUCache.put(1, 1); // cache is {1=1}
lRUCache.put(2, 2); // cache is {1=1, 2=2}
lRUCache.get(1);    // return 1
lRUCache.put(3, 3); // LRU key was 2, evicts key 2, cache is {1=1, 3=3}
lRUCache.get(2);    // returns -1 (not found)
lRUCache.put(4, 4); // LRU key was 1, evicts key 1, cache is {4=4, 3=3}
lRUCache.get(1);    // return -1 (not found)
lRUCache.get(3);    // return 3
lRUCache.get(4);    // return 4


Constraints:

* 1 <= capacity <= 3000
* 0 <= key <= 10^4
* 0 <= value <= 10^5
* At most 2 * 10^5 calls will be made to get and put.

# Python Dict Time Complexity Chart

| Operation      | Average Case | Worst Case |
|----------------|--------------|------------|
| Get Item       | O(1)         | O(n)       |
| Set Item       | O(1)         | O(n)       |
| Delete Item    | O(1)         | O(n)       |
| Iteration      | O(n)         | O(n)       |
| len()          | O(1)         | O(1)       |
| in (key check) | O(1)         | O(n)       |

# Implementation Details

If we were to only perform insert operations, we can easily get the most recent and least recent insert using a circular buffer. However, we need to allow for updates as well. Therefore, in order to keep our relative ordering (from most recently used to least recently used), we will use a linked list. The head of the linked list will be the most recently used, and the tail of the linked list will be the least recently used.

With this linked list, we will not traverse it in order to update our values. We used a linked list in order to maintain a relative ordering. In order to quickly access the elements in the linked list, we will use a dictionary. Whenever we get an item, we must use the dictionary to get the value (if it exists), and update the element's position in the linked list (which can be done in $\mathcal{O}(1)$) since we will maintain pointers to the head and tail of the list. Whenever we put an item, we once again update its position in the linked list, and we also remove and update the tail of necessary.

In [1]:
from typing import *

In [2]:
class KeyNode:
 """
 A key node will be stored in the hash map of the LRU cache. The value of the node is the value associated with the key
 in the hash map. However, we also maintain previous and next pointers in order to maintain a relative ordering
 between keys in the hash map. In particular, the previous key should be more recently used that this node, and the
 next key should be less recently used. The previous key should only be None if this node is the most recently used,
 and the next key should only be None if this node is the least recently used.
 """
 def __init__(self, val: int, prev_key: Union[int, None], next_key: Union[int, None]):
  self.val: int = val
  self.prev: Union[int, None] = prev_key
  self.next: Union[int, None] = next_key


class LRUCache:

 def __init__(self, capacity: int):
  assert capacity > 0
  self.key_map: Dict[int, KeyNode] = {} # hash map holding key nodes
  self.lru = self.mru = None  # holds the most and least recently used keys
  self.capacity = capacity # maximum capacity in the cache

 def _update_mru(self, key: int, curr_el: KeyNode):
    """
    Updates the mru in the hash_map
    :param key:         Key of the element that should be the new mru
    :param curr_el:     Corresponding value of this key in the key map, i.e., curr_el == self.hash_map[key]
    :return:
    """
    # this element should now point to the current mru
    curr_el.next = self.mru

    # previous mru should point back to this element
    self.key_map[self.mru].prev = key

    # now set this element as the new mru
    self.mru = key

 def _update_existing_element(self, key: int, curr_el: KeyNode):
  """
  Update the hash map so that this key and its corresponding element
  are now the new most recently used element.
  :param key:      Key in the key_map that should be the new mru.
  :param curr_el:  Corresponding value of this key in the key map, i.e., curr_el == self.hash_map[key]
  :return:
  """

  if key == self.mru:
   # is already mru, return
   return

  # The previous node should now point to this current element's next node.
  # Note that the current node is not the mru, thus it is impossible for its previous key to be None. However,
  # its next key may be None depending on if it is the lru.
  self.key_map[curr_el.prev].next = curr_el.next

  if key == self.lru:
    # The previous element should be the new lru. The previous element
    # already had its next pointer set to None from the line above
    self.lru = curr_el.prev
  else:
    # This element was not the lru, so its next element must exist, and we must update its previous pointer.
    self.key_map[curr_el.next].prev = curr_el.prev

  # update the mru in the cache
  self._update_mru(key, curr_el)

  # as new mru, prev should be None
  curr_el.prev = None

 def _insert_new_element(self, key: int, value: int):
  """
  Assuming the hash map has capacity, inserts a new element into the hash map and updates the relative ordering
  accordingly.
  :param key:   Key to use for the new element.
  :param value: Value associated with the new element.
  """
  # create the new node with next pointing to mru (mru may be None if this is the first element)
  curr_el = self.key_map[key] = KeyNode(value, None, None)

  # if this is the first new element, we must initialize mru and lru and return
  if len(self.key_map) == 1:
   self.mru = key
   self.lru = key
   return

  # update the mru in the cache
  self._update_mru(key, curr_el)

 def get(self, key: int) -> int:
  """
  Gets the value in the cache given a key. Returns -1 if the key does not exist.
  :param key: Key to index in the cache
  :return:    Key's corresponding value or -1
  """
  curr_el = self.key_map.get(key, None)

  # this element does not exist
  if curr_el is None:
   return -1

  # update the hash-map appropriately in case mru and/or lru get updated
  self._update_existing_element(key, curr_el)

  # finally, return the value
  return curr_el.val

 def put(self, key: int, value: int) -> None:
  curr_el = self.key_map.get(key, None)

  if curr_el is not None:
   # this element exists
   curr_el.val = value  # update its value

   # update the hash-map appropriately in case mru and/or lru get updated
   self._update_existing_element(key, curr_el)
  else:
   # this element does not exist

   if len(self.key_map) < self.capacity:
    # we have the capacity to store this element
    self._insert_new_element(key, value)

   else:
    # we must remove the lru element

    # get prev key of the lru element
    lru_prev = self.key_map[self.lru].prev

    # update lru's prev to point to None
    # Note that if the capacity is 1, this lru_prev may be None
    if lru_prev is not None:
     self.key_map[lru_prev].next = None

    # delete the current lru
    del self.key_map[self.lru]

    # update the prev to be the new lru
    if lru_prev is not None:
     self.lru = lru_prev
    else:
     # only happens when the capacity is 1
     self.lru = self.mru

    # we now have the capacity to store the new element
    self._insert_new_element(key, value)

 def get_keys(self, ordered=False):
  """
  Returns the keys in the LRU Cache as a list
  :param ordered: If true, the hash_map is traversed in order from most recent to least recent.
  :return keys:   List of keys in the LRU cache
  """
  if not ordered:
   # ordering does not matter, can simply return all keys
   return list(self.key_map.keys())
  else:
   # must traverse the hash map in order
   curr_key = self.mru
   keys = []
   while curr_key is not None:
    keys.append(curr_key)
    curr_key = self.key_map[curr_key].next
   return keys

# Your LRUCache object will be instantiated and called as such:
methods = ["LRUCache","put","put","put","put","get","get","get","get","put","get","get","get","get","get"]
parameters = [[3],[1,1],[2,2],[3,3],[4,4],[4],[3],[2],[1],[5,5],[1],[2],[3],[4],[5]]
obj = LRUCache(*parameters[0])
print(f"i: 0 | method: LRUCache | param: {parameters[0]}")
for i, method, param in zip(range(len(methods)), methods, parameters):
    if i == 0:
        continue
    print(f"i: {i} | method: {method} | param: {param}")
    method = getattr(obj, method)
    method(*param)
    print(f"mru after method: {obj.mru} | lru after method: {obj.lru}")
    print(f"Keys after operation: {obj.get_keys(False)}")
    print(f"Ordered Keys after operation: {obj.get_keys(True)}")
    print()

i: 0 | method: LRUCache | param: [3]
i: 1 | method: put | param: [1, 1]
mru after method: 1 | lru after method: 1
Keys after operation: [1]
Ordered Keys after operation: [1]

i: 2 | method: put | param: [2, 2]
mru after method: 2 | lru after method: 1
Keys after operation: [1, 2]
Ordered Keys after operation: [2, 1]

i: 3 | method: put | param: [3, 3]
mru after method: 3 | lru after method: 1
Keys after operation: [1, 2, 3]
Ordered Keys after operation: [3, 2, 1]

i: 4 | method: put | param: [4, 4]
mru after method: 4 | lru after method: 2
Keys after operation: [2, 3, 4]
Ordered Keys after operation: [4, 3, 2]

i: 5 | method: get | param: [4]
mru after method: 4 | lru after method: 2
Keys after operation: [2, 3, 4]
Ordered Keys after operation: [4, 3, 2]

i: 6 | method: get | param: [3]
mru after method: 3 | lru after method: 2
Keys after operation: [2, 3, 4]
Ordered Keys after operation: [3, 4, 2]

i: 7 | method: get | param: [2]
mru after method: 2 | lru after method: 4
Keys after op

In [3]:
class KeyNode:
 """
 A key node will be stored in the hash map of the LRU cache. The value of the node is the value associated with the key
 in the hash map. However, we also maintain previous and next pointers in order to maintain a relative ordering
 between keys in the hash map. In particular, the previous key should be more recently used that this node, and the
 next key should be less recently used. The previous key should only be None if this node is the most recently used,
 and the next key should only be None if this node is the least recently used.
 """
 def __init__(self, val: int, prev_key: Union[int, None], next_key: Union[int, None]):
  self.val: int = val
  self.prev: Union[int, None] = prev_key
  self.next: Union[int, None] = next_key


class LRUCache:

 def __init__(self, capacity: int):
  assert capacity > 0
  self.key_map: Dict[int, KeyNode] = {} # hash map holding key nodes
  self.lru = self.mru = None  # holds the most and least recently used keys
  self.capacity = capacity # maximum capacity in the cache

 def _update_existing_element(self, key: int, curr_el: KeyNode):
  """
  Update the hash map so that this key and its corresponding element
  are now the new most recently used element.
  :param key:      Key in the key_map that should be the new mru.
  :param curr_el:  Corresponding value of this key in the key map, i.e., curr_el == self.hash_map[key]
  :return:
  """

  if key == self.mru:
   # is already mru, return
   return

  # The previous node should now point to this current element's next node.
  # Note that the current node is not the mru, thus it is impossible for its previous key to be None. However,
  # its next key may be None.
  self.key_map[curr_el.prev].next = curr_el.next
  if curr_el.next:
   # this element exists and must point back
   self.key_map[curr_el.next].prev = curr_el.prev

  # this element was previously lru, so need to update lru
  if key == self.lru:
   self.lru = curr_el.prev

  # this element should now point to the current mru
  curr_el.next = self.mru

  # previous mru should point back to this element
  self.key_map[self.mru].prev = key

  # now set this element as the new mru
  self.mru = key

  # as new mru, prev should be None
  curr_el.prev = None

 def _insert_new_element(self, key: int, value: int):
  """
  Assuming the hash map has capacity, inserts a new element into the hash map and updates the relative ordering
  accordingly.
  :param key:   Key to use for the new element.
  :param value: Value associated with the new element.
  """
  # create the new node with next pointing to mru (mru may be None if this is the first element)
  self.key_map[key] = KeyNode(value, None, self.mru if self.mru is not None else None)

  # if this is the first new element, we must initialize mru and lru and return
  if len(self.key_map) == 1:
   self.mru = key
   self.lru = key
   return

  # update current mru to point back to this next element
  self.key_map[self.mru].prev = key

  # update this element to be the new mru
  self.mru = key

 def get(self, key: int) -> int:
  """
  Gets the value in the cache given a key. Returns -1 if the key does not exist.
  :param key: Key to index in the cache
  :return:    Key's corresponding value or -1
  """
  curr_el = self.key_map.get(key, None)

  # this element does not exist
  if curr_el is None:
   return -1

  # update the hash-map appropriately in case mru and/or lru get updated
  self._update_existing_element(key, curr_el)

  # finally, return the value
  return curr_el.val

 def put(self, key: int, value: int) -> None:
  curr_el = self.key_map.get(key, None)

  if curr_el is not None:
   # this element exists
   curr_el.val = value  # update its value

   # update the hash-map appropriately in case mru and/or lru get updated
   self._update_existing_element(key, curr_el)
  else:
   # this element does not exist
   if len(self.key_map) < self.capacity:
    # we have the capacity to store this element
    self._insert_new_element(key, value)

   else:
    # we must remove the lru element

    # get prev key of the lru element
    lru_prev = self.key_map[self.lru].prev

    # update lru's prev to point to None
    # Note that if the capacity is 1, this lru_prev may be None
    if lru_prev is not None:
     self.key_map[lru_prev].next = None

    # delete the current lru
    del self.key_map[self.lru]

    # update the prev to be the new lru
    self.lru = lru_prev

    # we now have the capacity to store the new element
    self._insert_new_element(key, value)

 def get_keys(self, ordered=False):
  """
  Returns the keys in the LRU Cache as a list
  :param ordered: If true, the hash_map is traversed in order from most recent to least recent.
  :return keys:   List of keys in the LRU cache
  """
  if not ordered:
   # ordering does not matter, can simply return all keys
   return list(self.key_map.keys())
  else:
   # must traverse the hash map in order
   curr_key = self.mru
   keys = []
   while curr_key is not None:
    keys.append(curr_key)
    curr_key = self.key_map[curr_key].next
   return keys